In [ ]:
import typing
import os

import hpotk
from phenopackets import Phenopacket

from genophenocorr.patient import PhenopacketPatientCreator
from genophenocorr.phenotype import PhenotypeCreator
from genophenocorr.protein import UniprotProteinMetadataService, ProteinAnnotationCache, ProtCachingFunctionalAnnotator
from genophenocorr.variant import VarCachingFunctionalAnnotator, VariantAnnotationCache, VepFunctionalAnnotator
from genophenocorr.cohort import PhenopacketCohortCreator


In [ ]:
fpath_hpo = 'hpo_data/hp.json'
cache_dir = 'testFiles/annotationsSub'
fpath_phenopackets = 'testFiles'
tx_id = 'NM_013275.6'
protein_id = 'Q15327'
if not os.path.isdir(cache_dir):
    os.mkdir(cache_dir)

In [ ]:
hpo: hpotk.ontology.MinimalOntology = hpotk.ontology.load.obographs.load_minimal_ontology(fpath_hpo)
validators = [
    hpotk.validate.AnnotationPropagationValidator(hpo),
    hpotk.validate.ObsoleteTermIdsValidator(hpo),
    hpotk.validate.PhenotypicAbnormalityValidator(hpo)
]
phenotype_creator = PhenotypeCreator(hpo, hpotk.validate.ValidationRunner(validators))

In [ ]:
# Functional annotator
vac = VariantAnnotationCache(cache_dir)
vep = VepFunctionalAnnotator()
vfa = VarCachingFunctionalAnnotator(vac, vep)

# Protein metadata
pm = UniprotProteinMetadataService()
pac = ProteinAnnotationCache(cache_dir)
pfa = ProtCachingFunctionalAnnotator(pac, pm)

# Assemble the patient creator
pc = PhenopacketPatientCreator(phenotype_creator, vfa, pfa)

In [ ]:
cc = PhenopacketCohortCreator(pc)

In [ ]:
patientCohort = cc.create_cohort(fpath_phenopackets)

In [ ]:
patientCohort.list_all_phenotypes()

In [ ]:
patientCohort.list_all_proteins(5)

In [ ]:
patientCohort.list_all_variants()

In [ ]:
patientCohort.list_all_patients()

In [ ]:
patientCohort.list_vartypes_by_tx('NM_013275.6')

In [ ]:
patientCohort.list_vartypes_by_tx()